In [ ]:
import requests
from datetime import datetime

In [ ]:
####################################################
#      捞单专用脚本
#    1. 填写 currentJob 值，目前只有 2 个值可选
#        DragForOrderPreSell 预售捞单
#        DragForOrder        非预售捞单
#    2. startStr 捞单开始时间，会自动向前扩大一天
#    3. endStr  捞单结束时间，会自动向后扩大一天
#    运行 Cell ==> Run All
#
#    脚本本质上对应的 Simba 的 2 个 job
#    预售   Simba job = RetrySplitOrderJob-presellorder-4hretry
#    非预售 Simba job = RetrySplitOrderJob-manual
#
####################################################


currentJob = 'DragForOrderPreSell'

startStr = '2019-06-02'
endStr   = '2019-06-12'

ssoid = '6543FDCA24B3558D6AB7447EEE922CE555E08A8936F70A5F54C7DF1EBBA87C6A'

In [ ]:
now = datetime.now()
start = datetime.strptime(startStr, '%Y-%m-%d')
end = datetime.strptime(endStr, '%Y-%m-%d')

s = (end - start).days * 24
e = (now - end).days * 24
e = e - 24 ## 结束时间向后扩一天
if e < 1:
    e = 1

s = s + 24  ## 开始时间向前扩一天

jobs = {
    'DragForOrderPreSell':{
        'menu_id': 32116,
        'commandLine':'java -jar /data/oms_workflow/oms-workflow.jar -j RetryComposeOrderJob -r {} -e {} -m false -b preSale -d prod',
        'releaseDesc':'捞单',
        'nodeId': 29502,
        'bizEndDate': '2018-06-11'
    },
    'DragForOrder':{
        'menu_id': 32114,
        'commandLine': 'java -jar /data/oms_workflow/oms-workflow.jar -j RetryComposeOrderJob -r {} -e {} -m false -d prod',
        'releaseDesc':'捞单',
        'nodeId': 29425,
        'bizEndDate': '2018-06-11'
    },
    'shippingRetry':{
        'menu_id': 34612,
        'commandLine': 'java -jar /data/oms_workflow/oms-workflow.jar -j AbnormalStatusOrderHandleJob -s {} -e {} -d prod',
    },
    'testJob':{
        'menu_id': 29260,
        'commandLine':'/data/sop/bin/java -Ddomain-prod -jar /data/supplier/apps/workflow/supplier-workflow.jar -j TestJob -d prod -s {}',
        'releaseDesc':'测试',
        'nodeId': 29524,
        'bizEndDate': '2018-06-11'
    }
}

## 替换 command line 参数
commandLine = jobs[currentJob]['commandLine'].format(s, e)
jobs[currentJob]['commandLine'] = commandLine

if not (jobs[currentJob]['menu_id'] 
    and jobs[currentJob]['commandLine']
    and jobs[currentJob]['releaseDesc']
    and jobs[currentJob]['nodeId']
    and jobs[currentJob]['bizEndDate']) or '{}' in jobs[currentJob]['commandLine']:
    raise ValueError('参数配置缺失')
    
cookies = {}
cookies['sessionId'] = ssoid

In [ ]:
### 获取 job 对比下数据

url = 'http://uwp.bigdata.beibei.com.cn/selfSelect/findFile?menu_id={}&type=1'.format(jobs[currentJob]['menu_id'])
res = requests.post(url, cookies=cookies)
commandLine = res.json()['map']['content']
diff = commandLine != jobs[currentJob]['commandLine']
print('commandLine is ' + str(diff))

In [ ]:
### 查询 job 列表

url = 'http://uwp.bigdata.beibei.com.cn/selfSelect/queryMenuBySearchNew'
params = {
    'project_id':21,
    'menu_name':'AbnormalStatusOrderHandleJob'
}

# res = requests.post(url, data=params, cookies=cookies)
# print(res.text)

In [ ]:
### 修改 job 信息
if diff:
    url = 'http://uwp.bigdata.beibei.com.cn/selfSelect/editFile'
    params = {
        'menu_id': jobs[currentJob]['menu_id'],
        'cluster_id': 1,
        'content': jobs[currentJob]['commandLine']
    }

    res = requests.post(url, data=params, cookies=cookies)
    print('execute edit job, result ' + res.json()['retMsg'])
else:
    print('command line euqals, skip edit job')
        

In [ ]:
### 发布修改后的 job
if diff:
    url = 'http://uwp.bigdata.beibei.com.cn/selfSelect/taskPublish'

    params = {
        'username': 'wayne.wang',
        'service': 'SelfSearch',
        'method': 'taskPublish',
        'menuId': jobs[currentJob]['menu_id'],
        'releaseDesc': jobs[currentJob]['releaseDesc']
    }
    res = requests.post(url, data=params, cookies=cookies)
    print('execute publish job, result ' + res.json()['retMsg'])
else:
    print('command line equals, skip publish job')

In [ ]:
### 试跑任务

url = 'http://uwp.bigdata.beibei.com.cn/taskList/trailRunning'

params = {
    'service': 'JobList',
    'method': 'trailRunning',
    'nodeId': jobs[currentJob]['nodeId'],
    'bizEndDate': jobs[currentJob]['bizEndDate']
}
res = requests.post(url, data=params, cookies=cookies)
print('execute trail running, result ' + res.json()['retMsg'])